In [2]:
# !pip install xgboost
# !pip install catboost

# Нашей задачей будет предсказать удовлетворённость пассажиров.

* Gender — пол пассажира (женский, мужской);
* Customer Type — тип клиента (постоянный/непостоянный клиент);
* Age — возраст клиента;
* Type of Travel — цель перелета (личная/деловая поездка);
* Class — туристический класс пассажира (Business, Eco, Eco Plus);
* Flight distance — расстояние полета;
* Inflight wifi service — уровень удовлетворённости Wi-Fi (0 — не применимо, 1–5);
* Departure/Arrival time convenient — уровень удовлетворённости временем отправления и прибытия;
* Ease of Online booking — уровень удовлетворённости онлайн-бронированием;
* Gate location — уровень удовлетворённости расположением выхода на посадку;
* Food and drink — уровень удовлетворённости едой и напитками;
* Online boarding — уровень удовлетворённости онлайн-регистрацией;
* Seat comfort — уровень удовлетворённости комфортом сидений;
* Inflight entertainment — уровень удовлетворённости развлечениями на борту;
* On-board service — уровень удовлетворённости сервисом на борту;
* Leg room service — уровень удовлетворённости местом для ног;
* Baggage handling — уровень удовлетворённости обработкой багажа;
* Check-in service — уровень удовлетворённости услугами регистрации;
* Inflight service — уровень удовлетворённости обслуживанием во время полёта;
* Cleanliness — уровень удовлетворённости чистотой;
* Departure Delay in Minutes — задержка при отправлении (в минутах);
* Arrival Delay in Minutes — задержка при прибытии (в минутах);
* Satisfaction — удовлетворённость авиакомпанией — целевая переменная (satisfaction/neutral/dissatisfaction).

In [28]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

import xgboost as xgb
import catboost as cb
from catboost import Pool
from catboost.utils import get_confusion_matrix

In [5]:
df = pd.read_csv('data/AirPass.csv')
df = df.drop(df.columns[[0]], axis=1)
df.head()

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [6]:
df.isnull().sum().sum()

310

### Задание 6.2

Теперь давайте избавимся от найденных пропусков. Заполните их все медианными значениями. После этого вычислите среднее арифметическое для признака, отражающего задержку при прибытии в минутах. Ответ округлите до двух знаков после точки-разделителя.

In [8]:
df['Arrival Delay in Minutes'] = df['Arrival Delay in Minutes'].fillna(df['Arrival Delay in Minutes'].median())
round(df['Arrival Delay in Minutes'].mean(), 2)

15.13

### Задание 6.3

Проведём небольшой разведывательный анализ. Посмотрим, в каких категориях пассажиров превалировали удовлетворённые полетом клиенты.

In [9]:
df.groupby('Gender')['satisfaction'].value_counts()*100/df.shape[0]

Gender  satisfaction           
Female  neutral or dissatisfied    29.058554
        satisfied                  21.687327
Male    neutral or dissatisfied    27.608177
        satisfied                  21.645942
Name: satisfaction, dtype: float64

In [10]:
df.groupby('Type of Travel')['satisfaction'].value_counts()*100/df.shape[0]

Type of Travel   satisfaction           
Business travel  satisfied                  40.177472
                 neutral or dissatisfied    28.785225
Personal Travel  neutral or dissatisfied    27.881506
                 satisfied                   3.155798
Name: satisfaction, dtype: float64

In [11]:
df.groupby('Class')['satisfaction'].value_counts()*100/df.shape[0]

Class     satisfaction           
Business  satisfied                  33.184478
          neutral or dissatisfied    14.614452
Eco       neutral or dissatisfied    36.614567
          satisfied                   8.374076
Eco Plus  neutral or dissatisfied     5.437712
          satisfied                   1.774715
Name: satisfaction, dtype: float64

In [12]:
# Перекодируем часть признаков
df['satisfaction'] = df['satisfaction'].map({'neutral or dissatisfied':0 , 'satisfied':1})
df['Customer Type'] = df['Customer Type'].map({'Loyal Customer':1, 'disloyal Customer':0})
df['Type of Travel'] = df['Type of Travel'].map({'Personal Travel':0, 'Business travel':1})
df['Gender'] = df['Gender'].map({'Male': 0, 'Female': 1})

In [13]:
df=pd.get_dummies(df)
df.shape

(103904, 26)

In [15]:
X = df.drop('satisfaction', axis=1)
y = df['satisfaction']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=26)
y_test.shape

(20781,)

### Задание 6.6

Теперь нам необходимо реализовать масштабирование данных. Для этого обучите на обучающей выборке метод StandardScaler() и с помощью него преобразуйте и обучающую, и тестовую выборки. Не забудьте, что целевую переменную обрабатывать не нужно.

**Примечание.** Отметим, что если бы дальше мы работали только с деревьями, масштабирование бы не требовалось. Однако мы реализуем его, чтобы можно было обучать и другие модели и сравнивать полученные результаты.

В качестве ответа введите самое первое значение из матрицы преобразованных признаков тестовой выборки. Округлите значение до двух знаков после точки-разделителя.

In [18]:
scaler = StandardScaler()
scaler.fit(X_train) 
X_train = scaler.transform(X_train) 
X_test = scaler.transform(X_test) 
round(X_test[0][0], 2)

0.94

### Задание 6.7

Перейдём к обучению моделей. В качестве первой модели возьмём самую простую — логистическую регрессию. Мы делаем это для того, чтобы потом сравнивать с ней полученные результаты: так вы сможете выяснить, насколько ансамбли смогут улучшить точность прогноза.

Обучите логистическую регрессию с параметрами по умолчанию на наших данных. В качестве ответа введите значение метрики f1_score. Ответ округлите до трёх знаков после точки-разделителя.

In [22]:
model_logreg = LogisticRegression()
model_logreg.fit(X_train, y_train)
preds_test_logreg = model_logreg.predict(X_test)
f1_score(y_test, preds_test_logreg)

0.8546883773161146

### Задание 6.8

Теперь перейдём к бустингу. Начнём с обучения первой модели — AdaBoost. В качестве базовой модели для неё возьмите решающее дерево с параметром random_state = 26.

Обучите AdaBoost, зафиксировав random_state со значением 26 и задав темп обучения 0.01. В качестве ответа введите значение метрики f1_score. Ответ округлите до трёх знаков после точки-разделителя.

In [23]:
model_ada = AdaBoostClassifier(DecisionTreeClassifier(random_state=26), random_state=26, learning_rate=0.01)

model_ada.fit(X_train, y_train)
preds_test_ada = model_ada.predict(X_test)
f1_score(y_test, preds_test_ada)

0.9404794558121674

### Задание 6.9

Перейдем к следующему алгоритму — градиентному бустингу.

Будем настраивать количество деревьев и темп обучения, делая перебор по следующей сетке:

params = {"n_estimators":2**np.arange(8), "learning_rate":0.1**np.arange(3)}
Используйте для поиска оптимальных параметров GridSearchCV, а для ускорения работы алгоритма задайте параметр кросс-валидации, равный 3.

Какое наибольшее значение метрики f1_score получилось? Ответ округлите до трёх знаков после точки-разделителя.

In [26]:
model_gradboos = GradientBoostingClassifier()
params = {"n_estimators":2**np.arange(8), "learning_rate":0.1**np.arange(3)}
gradboos = GridSearchCV(model_gradboos, 
                  params, 
                  cv=3, 
                  scoring=make_scorer(f1_score),
                  verbose=3)
 
gradboos.fit(X_train, y_train)
 
print("Лучшие гиперпараметры:", gradboos.best_params_)
print("Лучшее значение метрики:", gradboos.best_score_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV 1/3] END .learning_rate=1.0, n_estimators=1;, score=0.873 total time=   0.1s
[CV 2/3] END .learning_rate=1.0, n_estimators=1;, score=0.870 total time=   0.1s
[CV 3/3] END .learning_rate=1.0, n_estimators=1;, score=0.871 total time=   0.0s
[CV 1/3] END .learning_rate=1.0, n_estimators=2;, score=0.880 total time=   0.1s
[CV 2/3] END .learning_rate=1.0, n_estimators=2;, score=0.878 total time=   0.1s
[CV 3/3] END .learning_rate=1.0, n_estimators=2;, score=0.875 total time=   0.1s
[CV 1/3] END .learning_rate=1.0, n_estimators=4;, score=0.901 total time=   0.4s
[CV 2/3] END .learning_rate=1.0, n_estimators=4;, score=0.896 total time=   0.4s
[CV 3/3] END .learning_rate=1.0, n_estimators=4;, score=0.897 total time=   0.4s
[CV 1/3] END .learning_rate=1.0, n_estimators=8;, score=0.920 total time=   0.8s
[CV 2/3] END .learning_rate=1.0, n_estimators=8;, score=0.920 total time=   0.8s
[CV 3/3] END .learning_rate=1.0, n_estimators=8;

### Задание 6.10

Обучите алгоритм XGBoost. Так как он достаточно мощный «из коробки», определите его с параметрами по умолчанию, только задайте random_state = 26. Какое значение метрики f1_score получилось? Ответ округлите до трёх знаков после точки-разделителя.

**Подсказка.** Для того чтобы обучить алгоритм XGBoost для решения задачи классификации, вам понадобится XGBClassifier из библиотеки xgboost, установленной ранее. Вся дальнейшая последовательность действий (обучение модели, предсказание, оценка качества) идентична другим алгоритмам, например логистической регрессии.

In [29]:
model_xgb = xgb.XGBClassifier(random_state = 26)
model_xgb.fit(X_train,y_train)
preds_test_xgb = model_xgb.predict(X_test)
f1_score(y_test, preds_test_xgb)

0.9579785161685312

### Задание 6.11

Обучите алгоритм CatBoost. Как и XGBoost, будем обучать его с настройками по умолчанию и заданным random_state = 26. Какое значение метрики f1_score получилось? Ответ округлите до трёх знаков после точки-разделителя.

**Подсказка.** Для того чтобы обучить алгоритм CatBoost, вам понадобится CatBoostClassifier() из библиотеки catboost, установленной ранее. Вся дальнейшая последовательность действий (обучение модели, предсказание, оценка качества) идентична другим алгоритмам, например логистической регрессии.

In [36]:
model_cat = cb.CatBoostClassifier(random_state = 42)
model_cat.fit(X_train, y_train)
preds_class_cat = model_cat.predict(X_test)
f1_score(y_test, preds_class_cat)

Learning rate set to 0.068023
0:	learn: 0.6018174	total: 19.3ms	remaining: 19.3s
1:	learn: 0.5281923	total: 36.3ms	remaining: 18.1s
2:	learn: 0.4531336	total: 53.6ms	remaining: 17.8s
3:	learn: 0.3921318	total: 70.3ms	remaining: 17.5s
4:	learn: 0.3557526	total: 86.9ms	remaining: 17.3s
5:	learn: 0.3313638	total: 102ms	remaining: 16.9s
6:	learn: 0.3096950	total: 118ms	remaining: 16.8s
7:	learn: 0.2904554	total: 134ms	remaining: 16.7s
8:	learn: 0.2729980	total: 151ms	remaining: 16.6s
9:	learn: 0.2584682	total: 167ms	remaining: 16.5s
10:	learn: 0.2422868	total: 187ms	remaining: 16.8s
11:	learn: 0.2299315	total: 203ms	remaining: 16.7s
12:	learn: 0.2190787	total: 220ms	remaining: 16.7s
13:	learn: 0.2077445	total: 236ms	remaining: 16.6s
14:	learn: 0.2021987	total: 252ms	remaining: 16.6s
15:	learn: 0.1958336	total: 268ms	remaining: 16.5s
16:	learn: 0.1917574	total: 284ms	remaining: 16.4s
17:	learn: 0.1875629	total: 298ms	remaining: 16.2s
18:	learn: 0.1841534	total: 313ms	remaining: 16.2s
19:	le

0.9602623242372034

In [37]:
get_confusion_matrix(model_cat, Pool(X_train, y_train))

array([[46665.,   541.],
       [ 1249., 34668.]])

### Задание 6.13

Оцените важность признаков для модели из предыдущего задания. Отметьте признак, который оказывает наибольшее влияние на значение целевой переменной:

In [40]:

len(model_cat.get_feature_importance())

25

In [41]:
len(df.columns)

26

In [47]:
X.columns

Index(['id', 'Gender', 'Customer Type', 'Age', 'Type of Travel',
       'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes',
       'Class_Business', 'Class_Eco', 'Class_Eco Plus'],
      dtype='object')

In [48]:
pd.DataFrame({'feature_importance': model_cat.get_feature_importance(), 
              'feature_names':X.columns}).sort_values(by=['feature_importance'], 
                                                           ascending=False)

,feature_importance,feature_names
6,25.767503,Inflight wifi service
4,17.740528,Type of Travel
11,7.177993,Online boarding
2,6.656839,Customer Type
22,5.743438,Class_Business
17,3.953178,Checkin service
3,3.668340,Age
9,3.555152,Gate location
16,3.519532,Baggage handling
12,3.315855,Seat comfort
